<a href="https://colab.research.google.com/github/suyogpradhan0/Spatial-Data-Management-with-Google-Earth-Engine/blob/main/Cloud_free_imagery_of_Knox_County%2C_Tennessee_with_Landsat_9%2C_Sentinel_2_and_NAIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-414/blob/master/book/labs/lab_04.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

## Datasets

The datasets being used in the lab are listed below:

- [TIGER: US Census Counties](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties)
- [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)
- [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
- [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ)

## Question 1

Write a program to find out how many counties are named `Knox` in the US.

In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

In [3]:
# Load TIGER/Line dataset for US Census Counties
counties = ee.FeatureCollection('TIGER/2018/Counties')

# Filter counties named "Knox"
knox_counties = counties.filter(ee.Filter.eq('NAME', 'Knox'))

# Get the number of Knox counties
num_knox_counties = knox_counties.size().getInfo()

print(f'Number of counties named "Knox" in the US: {num_knox_counties}')

Number of counties named "Knox" in the US: 9


![](https://i.imgur.com/3Jg9P6X.png)

## Question 2

Display Knox county of Tennesse with outline only (no fill color) on the map. (Hint: The `STATEFP` of Tennessee is `47`)

In [4]:
# Filter Knox County in Tennessee
knox_county = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Knox'),
    ee.Filter.eq('STATEFP', '47')  # State code for Tennessee
)).geometry()

# Create a Map object
Map = geemap.Map()

# Display Knox County with outline only (no fill color)
Map.addLayer(knox_county, {'color': 'FF000000', 'fillColor': 'transparent', 'width': 2}, 'Knox County Outline')

# Center the map around Knox County
Map.centerObject(knox_county, 10)

#Add text to the map
text = "Made by Suyog Pradhan"
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/28Iaw9b.png)

## Question 3

Use [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2) data to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [5]:
# Load Landsat-9 Collection 2, Tier 1, Level 2 (C02/T1_L2) data
landsat_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

# Filter Landsat-9 data for Knox County
knox_landsat = landsat_collection.filterBounds(knox_county)

# Function to mask clouds and cloud shadows
def mask_clouds(image):
        qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)

        # Apply the scaling factors to the appropriate bands.
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

        # Replace the original bands with the scaled ones and apply the masks.
        return image.addBands(opticalBands, None, True).updateMask(qaMask)

# Apply cloud masking function to the Landsat-9 collection
cloud_free_landsat = knox_landsat.map(mask_clouds)

# Take the median of the cloud-free images to create a composite
composite = cloud_free_landsat.median()

# Define the visualization bands
visualization_bands = ['SR_B5', 'SR_B4', 'SR_B3']

# Clip the image to Knox County
landsat_image_clipped = composite.clip(knox_county)

# Display the cloud-free Landsat-9 image on the map
Map = geemap.Map()
Map.centerObject(knox_county, 10)
Map.addLayer(landsat_image_clipped, {'bands': visualization_bands, 'min': 0, 'max': 0.5}, 'Landsat-9 Image (Cloud-free)')

#Add text to the map
text = "Made by Suyog Pradhan"
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/m72amN0.png)

## Question 4

Use [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) data to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [6]:
# Load Sentinel-2 imagery for Knox County, Tennessee, and apply cloud masking
sentinel2_collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2022-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)

# Select bands for visualization (e.g., RGB)
visualization_bands = ['B4', 'B3', 'B2']

# Create a mosaic of the imagery
mosaic = sentinel2_collection.mosaic()

# Clip the mosaic to Knox County
mosaic_clipped = mosaic.clip(knox_county)

# Display the cloud-free imagery for Knox County with the specified band combination
Map = geemap.Map()
Map.centerObject(knox_county, 10)
Map.addLayer(knox_county, {'color': 'red'}, 'Knox County Outline')
Map.addLayer(mosaic_clipped, {'bands': visualization_bands, 'min': 0, 'max': 1000}, 'Sentinel-2 Image')
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/vM1M8Gc.png)

## Question 5

Use [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ) imagery to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [7]:
# Load NAIP imagery for Knox County
naip_collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(knox_county)
    .filterDate('2021-01-01', '2021-12-31')
    .mosaic()
    .clip(knox_county)
    .select('N', 'R', 'G')
)

# Function to mask clouds and shadows using cloud percentage threshold
def mask_clouds(image):
    # Select the bands 'R', 'G', 'B', and 'N'
    bands = ['N', 'R', 'G']
    # Calculate the cloud percentage for each band
    cloud_percentage = image.select(bands).reduce(ee.Reducer.allNonZero()).divide(image.select(bands).reduce(ee.Reducer.count())).multiply(100)
    # Create a mask for pixels with cloud percentage less than 5
    cloud_mask = cloud_percentage.lt(5)
    # Update the image mask
    return image.updateMask(cloud_mask)

# Apply cloud masking function to the NAIP collection
cloud_free_naip = naip_collection.map(mask_clouds)

# Take the median of the cloud-free images to create a composite
composite = cloud_free_naip.median()

# Define the visualization bands
visualization_bands = ['N', 'R', 'G']

# Display the NAIP imagery for Knox County
Map = geemap.Map()
Map.centerObject(knox_county, 10)
Map.addLayer(naip_collection, {'bands': visualization_bands, 'min': 0, 'max': 255}, 'NAIP Composite')
Map.add_text(text, fontsize=20, position='bottomright')
Map


Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/iZSGqGS.png)